In [25]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

DATA_DIR = Path('/home/parambrata-ghosh/Development/Personal/Projects/Food_Delivery_Startup/input/RPC_18_Datasets')

## Setup

# Data Cleaning

Clean and prepare datasets for analysis.

In [26]:
# Load dimension tables
df_customer = pd.read_csv(DATA_DIR / 'dim_customer.csv')
df_delivery_partner = pd.read_csv(DATA_DIR / 'dim_delivery_partner_.csv')
df_menu_item = pd.read_csv(DATA_DIR / 'dim_menu_item.csv')
df_restaurant = pd.read_csv(DATA_DIR / 'dim_restaurant.csv')

# Load fact tables
df_orders = pd.read_csv(DATA_DIR / 'fact_orders.csv')
df_order_items = pd.read_csv(DATA_DIR / 'fact_order_items.csv')
df_delivery_performance = pd.read_csv(DATA_DIR / 'fact_delivery_performance.csv')
df_ratings = pd.read_csv(DATA_DIR / 'fact_ratings.csv')

print(f"Loaded {df_customer.shape[0]:,} customers, {df_restaurant.shape[0]:,} restaurants, {df_delivery_partner.shape[0]:,} delivery partners")
print(f"Loaded {df_orders.shape[0]:,} orders, {df_ratings.shape[0]:,} ratings")

Loaded 107,776 customers, 19,995 restaurants, 15,000 delivery partners
Loaded 149,166 orders, 68,842 ratings


## Load Data

In [ ]:
print(f"Before cleaning: {len(df_ratings):,} ratings")

# remove null values
df_ratings_clean = df_ratings.dropna()
print(f"After removing nulls: {len(df_ratings_clean):,} ratings")

# remove duplicates
df_ratings_clean = df_ratings_clean.drop_duplicates()
print(f"After removing duplicates: {len(df_ratings_clean):,} ratings")

# update original dataframe
df_ratings = df_ratings_clean

Before cleaning: 68,842 ratings
After removing nulls: 68,825 ratings
After removing duplicates: 68,825 ratings


## Clean Ratings Table

In [28]:
missing_dp_orders = df_orders[df_orders['delivery_partner_id'].isna()]
print(f"Missing delivery partners: {len(missing_dp_orders):,}")
print(f"Cancelled orders: {(missing_dp_orders['is_cancelled'] == 'Y').sum():,}")
print(f"Non-cancelled orders: {(missing_dp_orders['is_cancelled'] == 'N').sum():,}")

Missing delivery partners: 5,635
Cancelled orders: 5,635
Non-cancelled orders: 0


## Investigate Missing Delivery Partners

In [29]:
df_orders['order_timestamp'] = pd.to_datetime(df_orders['order_timestamp'])
df_customer['signup_date'] = pd.to_datetime(df_customer['signup_date'], format='%d-%m-%Y')
df_ratings['review_timestamp'] = pd.to_datetime(df_ratings['review_timestamp'], format='%d-%m-%Y %H:%M')

print("Date columns converted successfully")

Date columns converted successfully


## Convert Date Columns

In [30]:
df_orders['month'] = df_orders['order_timestamp'].dt.to_period('M').astype(str)

monthly = (
    df_orders.groupby('month', as_index=False)
    .agg(
        total_orders=('order_timestamp', 'count'),
        cancelled_orders=('is_cancelled', lambda x: (x == 'Y').sum())
    )
)
monthly['monthly_change'] = monthly['total_orders'].diff()
monthly = monthly.sort_values('month')

monthly

,month,total_orders,cancelled_orders,monthly_change
0,2025-01,23539,1463,NaN
1,2025-02,22667,1395,-872.0
2,2025-03,23543,1410,876.0
3,2025-04,21466,1256,-2077.0
4,2025-05,22591,1370,1125.0
5,2025-06,9293,1074,-13298.0
6,2025-07,8818,1050,-475.0
7,2025-08,8555,1070,-263.0
8,2025-09,8694,1024,139.0


## Monthly Order Trends

In [ ]:
# merge orders with customer city
df_orders_city = df_orders.merge(df_customer[['customer_id', 'city']], on='customer_id', how='inner')

# filter non cancelled orders
df_orders_city = df_orders_city[df_orders_city['is_cancelled'] == 'N']

# add month column
df_orders_city['month'] = df_orders_city['order_timestamp'].dt.to_period('M').astype(str)

print(f"Total non-cancelled orders: {len(df_orders_city):,}")

Total non-cancelled orders: 138,054


## City-wise Order Analysis

In [32]:
monthly_city_orders = (
    df_orders_city.groupby(['city', 'month'], as_index=False)
    .agg(orders=('order_id', 'count'))
)

monthly_city_orders.head(10)

,city,month,orders
0,Ahmedabad,2025-01,1570
1,Ahmedabad,2025-02,1458
2,Ahmedabad,2025-03,1559
3,Ahmedabad,2025-04,1367
4,Ahmedabad,2025-05,1458
5,Ahmedabad,2025-06,553
6,Ahmedabad,2025-07,564
7,Ahmedabad,2025-08,488
8,Ahmedabad,2025-09,545
9,Bengaluru,2025-01,6104


In [33]:
pre_crisis_months = ['2025-01','2025-02','2025-03','2025-04','2025-05']
crisis_months = ['2025-06','2025-07','2025-08','2025-09']


## Pre-Crisis vs Crisis Comparison

In [34]:
pre_crisis_months = ['2025-01', '2025-02', '2025-03', '2025-04', '2025-05']
crisis_months = ['2025-06', '2025-07', '2025-08', '2025-09']

summary = (
    monthly_city_orders
    .groupby('city')
    .apply(lambda g: pd.Series({
        'avg_pre_crisis': g.loc[g['month'].isin(pre_crisis_months), 'orders'].mean(),
        'avg_crisis': g.loc[g['month'].isin(crisis_months), 'orders'].mean()
    }), include_groups=False)
    .reset_index()
)

In [35]:
summary['pct_change'] = (
    (summary['avg_crisis'] - summary['avg_pre_crisis']) /
     summary['avg_pre_crisis'] * 100
)

In [36]:
summary['pct_change'] = (
    (summary['avg_crisis'] - summary['avg_pre_crisis']) / summary['avg_pre_crisis'] * 100
)

summary = summary.sort_values('pct_change', ascending=True)
summary

,city,avg_pre_crisis,avg_crisis,pct_change
1,Bengaluru,6044.0,2160.50,-64.253805
6,Mumbai,3452.0,1238.25,-64.129490
5,Kolkata,1470.0,533.00,-63.741497
0,Ahmedabad,1482.4,537.50,-63.741230
7,Pune,1704.8,623.50,-63.426795
3,Delhi,2999.6,1101.50,-63.278437
4,Hyderabad,2096.2,773.00,-63.123748
2,Chennai,2133.4,818.25,-61.645730
